In [1]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/home/fabian/aistudy/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [3]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [4]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.4)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, 
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', 
                                                    target_size = (150, 150))     

validation_generator =  test_datagen.flow_from_directory( validation_dir, batch_size  = 20, class_mode  = 'binary', target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 80,
            validation_steps = 50,
            verbose = 1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/80
100/100 [==============================] - 16s 159ms/step - loss: 0.5073 - accuracy: 0.7520 - val_loss: 0.6073 - val_accuracy: 0.8430
Epoch 2/80
100/100 [==============================] - 13s 133ms/step - loss: 0.3578 - accuracy: 0.8400 - val_loss: 0.1935 - val_accuracy: 0.9430
Epoch 3/80
100/100 [==============================] - 13s 135ms/step - loss: 0.3534 - accuracy: 0.8480 - val_loss: 0.4894 - val_accuracy: 0.9020
Epoch 4/80
100/100 [==============================] - 13s 134ms/step - loss: 0.3407 - accuracy: 0.8610 - val_loss: 0.4122 - val_accuracy: 0.9220
Epoch 5/80
100/100 [==============================] - 13s 135ms/step - loss: 0.3237 - accuracy: 0.8655 - val_loss: 0.2386 - val_accuracy: 0.9560
Epoch 6/80
100/100 [==============================] - 13s 135ms/step - loss: 0.3116 - accuracy: 0.8705 - val_loss: 0.3874 - val_accuracy: 0.9390
Epoch 7/80
100/100 [=================

Epoch 56/80
100/100 [==============================] - 14s 135ms/step - loss: 0.1852 - accuracy: 0.9330 - val_loss: 0.4933 - val_accuracy: 0.9590
Epoch 57/80
100/100 [==============================] - 13s 134ms/step - loss: 0.2129 - accuracy: 0.9275 - val_loss: 0.5228 - val_accuracy: 0.9610
Epoch 58/80
100/100 [==============================] - 13s 132ms/step - loss: 0.1966 - accuracy: 0.9270 - val_loss: 0.5041 - val_accuracy: 0.9610
Epoch 59/80
100/100 [==============================] - 14s 136ms/step - loss: 0.1978 - accuracy: 0.9305 - val_loss: 0.4897 - val_accuracy: 0.9600
Epoch 60/80
 43/100 [===========>..................] - ETA: 6s - loss: 0.2135 - accuracy: 0.9337

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc = 'upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc = 'upper left')
plt.show()